In [1]:
import findspark
findspark.init() 

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession
    .builder
    .appName('Flights')
    .master('local[*]')
    .getOrCreate()
)

21/12/06 14:52:25 WARN Utils: Your hostname, chabrik-VM resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/12/06 14:52:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/06 14:52:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

#path = ""

df = spark.read.format("csv").option("header","true").load("/home/chabrik/Downloads/DATA/flights.csv")
df = df.drop('FLIGHT_NUMBER','TAIL_NUMBER','DEPARTURE_TIME''TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME','TAXI_OUT','ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','SCHEDULED_ARRIVAL','ARRIVAL_TIME','ARRIVAL_DELAY' ,'DIVERTED','CANCELLED','CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'  )
#df.show()

In [4]:
df.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'DISTANCE']

In [5]:
for c in df.columns:
    df = df.na.drop(subset=c)


In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_NUMERIC").fit(df) for column in ['AIRLINE', 'ORIGIN_AIRPORT','DESTINATION_AIRPORT']]

pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df).transform(df).drop('AIRLINE', 'ORIGIN_AIRPORT','DESTINATION_AIRPORT')


In [7]:
from pyspark.sql.types import IntegerType
for col in df_indexed.columns:
     df_indexed = df_indexed.withColumn(col, df_indexed[col].cast(IntegerType()))

In [8]:
feature_columns = ['YEAR','MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'SCHEDULED_DEPARTURE', 'DISTANCE', 'AIRLINE_NUMERIC', 'ORIGIN_AIRPORT_NUMERIC', 'DESTINATION_AIRPORT_NUMERIC']

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

dataset = assembler.transform(df_indexed)
train, test = dataset.randomSplit([0.7, 0.3])


In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

model = RandomForestRegressor(featuresCol="features", labelCol='DEPARTURE_DELAY')


model = model.fit(train)


21/12/06 14:56:37 WARN MemoryStore: Not enough space to cache rdd_49_0 in memory! (computed 61.4 MiB so far)
21/12/06 14:56:37 WARN BlockManager: Persisting block rdd_49_0 to disk instead.
21/12/06 14:57:03 WARN MemoryStore: Not enough space to cache rdd_49_1 in memory! (computed 61.4 MiB so far)
21/12/06 14:57:03 WARN BlockManager: Persisting block rdd_49_1 to disk instead.
21/12/06 14:57:21 WARN MemoryStore: Not enough space to cache rdd_49_2 in memory! (computed 61.4 MiB so far)
21/12/06 14:57:21 WARN BlockManager: Persisting block rdd_49_2 to disk instead.
21/12/06 14:57:39 WARN MemoryStore: Not enough space to cache rdd_49_3 in memory! (computed 61.4 MiB so far)
21/12/06 14:57:39 WARN BlockManager: Persisting block rdd_49_3 to disk instead.
21/12/06 14:57:55 WARN MemoryStore: Not enough space to cache rdd_49_4 in memory! (computed 61.4 MiB so far)
21/12/06 14:57:55 WARN BlockManager: Persisting block rdd_49_4 to disk instead.
21/12/06 14:58:02 WARN MemoryStore: Not enough space to

In [10]:
#model.save( 'model_saveddd')


In [11]:
import matplotlib.pyplot as plt
import pandas as pd

predictions = model.transform(test)
evaluator = RegressionEvaluator(labelCol="DEPARTURE_DELAY", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(rmse)
rfResult = predictions.toPandas()


In [12]:
rfResult.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,DISTANCE,AIRLINE_NUMERIC,ORIGIN_AIRPORT_NUMERIC,DESTINATION_AIRPORT_NUMERIC,features,prediction
0,2015,1,1,4,25,24,-1,1448,9,10,66,"[2015.0, 1.0, 1.0, 4.0, 25.0, 25.0, 1448.0, 9....",2.274405
1,2015,1,1,4,30,33,3,1747,1,8,0,"[2015.0, 1.0, 1.0, 4.0, 30.0, 30.0, 1747.0, 1....",3.536336
2,2015,1,1,4,30,44,14,2125,8,4,14,"[2015.0, 1.0, 1.0, 4.0, 30.0, 30.0, 2125.0, 8....",2.917003
3,2015,1,1,4,35,35,0,1535,1,4,9,"[2015.0, 1.0, 1.0, 4.0, 35.0, 35.0, 1535.0, 1....",2.723672
4,2015,1,1,4,45,41,-4,1448,9,65,10,"[2015.0, 1.0, 1.0, 4.0, 45.0, 45.0, 1448.0, 9....",1.929071


In [13]:
'''
plt.plot(rfResult.DEPARTURE_DELAY, rfResult.prediction, 'bo')
plt.xlabel('delay')
plt.ylabel('Prediction')
plt.suptitle("Model Performance RMSE: %f" % rmse)
plt.show()
'''

'\nplt.plot(rfResult.DEPARTURE_DELAY, rfResult.prediction, \'bo\')\nplt.xlabel(\'delay\')\nplt.ylabel(\'Prediction\')\nplt.suptitle("Model Performance RMSE: %f" % rmse)\nplt.show()\n'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import *
'''
def process_row(df, epoch_id):
    db_target_url = "jdbc:mysql://localhost:3306/Streaming"
    db_target_properties = {"user":"root", "password":""}
    df.write.jdbc(url=db_target_url, table="Result", mode="append", properties=db_target_properties)
    pass
'''
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 8990).load()

#line = lines.select(split(lines.value,','))

flight = lines.select(split(lines.value, ",").getItem(0).alias("YEAR"),split(lines.value, ",").getItem(1).alias("MONTH"),split(lines.value, ",").getItem(2).alias("DAY"),split(lines.value, ",").getItem(3).alias("DAY_OF_WEEK"),split(lines.value, ",").getItem(4).alias("AIRLINE"),split(lines.value, ",").getItem(5).alias("FLIGHT_NUMBER"),split(lines.value, ",").getItem(6).alias("TAIL_NUMBER"),split(lines.value, ",").getItem(7).alias("ORIGIN_AIRPORT"),split(lines.value, ",").getItem(8).alias("DESTINATION_AIRPORT"),split(lines.value, ",").getItem(9).alias("SCHEDULED_DEPARTURE"),split(lines.value, ",").getItem(10).alias("DEPARTURE_TIME"),split(lines.value, ",").getItem(11).alias("DEPARTURE_DELAY"),split(lines.value, ",").getItem(12).alias("TAXI_OUT"),split(lines.value, ",").getItem(13).alias("WHEELS_OFF"),split(lines.value, ",").getItem(14).alias("SCHEDULED_TIME"),split(lines.value, ",").getItem(15).alias("ELAPSED_TIME"),split(lines.value, ",").getItem(16).alias("AIR_TIME"),split(lines.value, ",").getItem(17).alias("DISTANCE"),split(lines.value, ",").getItem(18).alias("WHEELS_ON"),split(lines.value, ",").getItem(19).alias("TAXI_IN"),split(lines.value, ",").getItem(20).alias("SCHEDULED_ARRIVAL"),split(lines.value, ",").getItem(21).alias("ARRIVAL_TIME"),split(lines.value, ",").getItem(22).alias("ARRIVAL_DELAY"),split(lines.value, ",").getItem(23).alias("DIVERTED"),split(lines.value, ",").getItem(24).alias("CANCELLED"),split(lines.value, ",").getItem(25).alias("CANCELLATION_REASON"),split(lines.value, ",").getItem(26).alias("AIR_SYSTEM_DELAY"),split(lines.value, ",").getItem(27).alias("SECURITY_DELAY"),split(lines.value, ",").getItem(28).alias("AIRLINE_DELAY"),split(lines.value, ",").getItem(29).alias("LATE_AIRCRAFT_DELAY"),split(lines.value, ",").getItem(30).alias("WEATHER_DELAY"))

flight = flight.drop('FLIGHT_NUMBER','TAIL_NUMBER','DEPARTURE_TIME''TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME','TAXI_OUT','ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','SCHEDULED_ARRIVAL','ARRIVAL_TIME','ARRIVAL_DELAY' ,'DIVERTED','CANCELLED','CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'  )

flight = pipeline.fit(flight).transform(flight).drop('AIRLINE', 'ORIGIN_AIRPORT','DESTINATION_AIRPORT')

for col in flight.columns:
     flight = flight.withColumn(col, flight[col].cast(IntegerType()))

flight = assembler.transform(flight)
flight = model.transform(flight)
query = flight.writeStream.format("console").start()
#query = flight.writeStream.foreachBatch(process_row).start()
#predictions = model.transform(line)


query.awaitTermination()

21/12/06 15:25:17 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
21/12/06 15:25:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2ee118e9-4f56-4457-9662-ce7e3b49367e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/06 15:25:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------+----------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|features|prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------+----------+
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------+----------+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+-------------

-------------------------------------------
Batch: 8
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|        prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|2015|    1|  1|          4|                 30|            44|             14|    2125|              8|                     4|                         14|[2015.0,1.0,1.0,4...|2.9170026733259498|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+---------------

-------------------------------------------
Batch: 16
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|        prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|2015|    1|  1|          4|                 45|            41|             -4|    1448|              9|                    65|                         10|[2015.0,1.0,1.0,4...|1.9290708783658832|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+--------------

-------------------------------------------
Batch: 24
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|        prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|2015|    1|  1|          4|                105|           103|             -2|    2342|              2|                     4|                         24|[2015.0,1.0,1.0,4...|2.7236719350585408|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+--------------

...

-------------------------------------------
Batch: 456
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+-----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|       prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+-----------------+
|2015|    1|  1|          4|                611|           607|             -4|    1237|              7|                    13|                         21|[2015.0,1.0,1.0,4...|2.274405315379867|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+-----------------

-------------------------------------------
Batch: 464
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+-----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|       prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+-----------------+
|2015|    1|  1|          4|                615|           605|            -10|     601|              8|                    74|                          6|[2015.0,1.0,1.0,4...|2.503751221884732|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+-----------------

-------------------------------------------
Batch: 472
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|      prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+----------------+
|2015|    1|  1|          4|                615|           618|              3|     852|              2|                     9|                          2|[2015.0,1.0,1.0,4...|3.38640012706369|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+---------------------

-------------------------------------------
Batch: 480
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|        prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+------------------+
|2015|    1|  1|          4|                615|           609|             -6|     405|              1|                    22|                         12|[2015.0,1.0,1.0,4...|2.6069267238280553|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+-------------

-------------------------------------------
Batch: 488
-------------------------------------------
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+-----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|DISTANCE|AIRLINE_NUMERIC|ORIGIN_AIRPORT_NUMERIC|DESTINATION_AIRPORT_NUMERIC|            features|       prediction|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+----------------------+---------------------------+--------------------+-----------------+
|2015|    1|  1|          4|                615|           623|              8|     925|              4|                    78|                          7|[2015.0,1.0,1.0,4...|2.702931516844764|
+----+-----+---+-----------+-------------------+--------------+---------------+--------+---------------+-----------------

In [ ]:
query.stop()